<a href="https://colab.research.google.com/github/jongwoonalee/jongwoonalee.github.io/blob/main/Additinal_Project_Using_Differnt_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Panorama Creation with Advanced Feature Matching Metrics
1. This notebook implemented a robust panorama creation pipeline using various feature matching metrics and advanced blending techniques.

##  Table of Contents
1. [Introduction and Setup](#introduction)
2. [Image Loading and Preprocessing](#loading)
3. [Feature Detection and Description](#features)
4. [Advanced Feature Matching](#matching)
5. [Homography Estimation and RANSAC](#ransac)
6. [Image Warping and Blending](#warping)
7. [Results and Analysis](#results)

##  1. Introduction and Setup <a name="introduction"></a>

### Overview
 We explored beyond traditional methods by implementing and comparing multiple distance metrics for feature matching.

### Key Innovations
- **Multi-scale feature detection** for improved coverage
- **Advanced distance metrics** for robust matching
- **Attention-based matching** inspired by transformer architectures
- **Adaptive RANSAC** with automatic threshold adjustment
- **Smooth blending** with distance transform-based feathering

### Distance Metrics Explored
1. **Chi-squared distance** - Effective for histogram comparisons
2. **Bhattacharyya distance** - Measures similarity between probability distributions
3. **Earth Mover's Distance (Wasserstein)** - Robust to small distribution shifts
4. **Attention-based matching** - Uses global context for better correspondences
5. **Traditional metrics** - Euclidean and correlation-based matching

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from scipy.ndimage import maximum_filter, distance_transform_edt
from scipy.signal import convolve2d
from scipy.spatial.distance import cdist
from scipy.stats import wasserstein_distance

# Set matplotlib parameters for better visualization
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['image.cmap'] = 'gray'

# Configuration
CONFIG = {
    'folder_path': '/Volumes/SSD 1T/downloads_2024_iMAC/science_library_2',
    'max_images': 18,  # Limit number of images for processing
    'output_filename': 'panorama_advanced_metrics.jpg',
    'feature_params': {
        'max_corners': 1500,
        'grid_size': 40,
        'window_size': 16
    },
    'matching_params': {
        'threshold': 0.7,
        'attention_threshold': 0.3
    },
    'ransac_params': {
        'inlier_threshold': 4.0,
        'iterations': 2000
    }
}

Verify Setup

In [ ]:
# Check if folder exists
if os.path.exists(CONFIG['folder_path']):
    print(f"✓ Folder found: {CONFIG['folder_path']}")

    # List image files
    image_files = [f for f in os.listdir(CONFIG['folder_path'])
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"✓ Found {len(image_files)} image files")

    if image_files:
        print("\nFirst 5 images:")
        for i, img in enumerate(image_files[:5]):
            print(f"  {i+1}. {img}")
else:
    print(f"✗ Folder not found: {CONFIG['folder_path']}")
    print("Please update CONFIG['folder_path'] with the correct path to your images")

##  2. Image Loading and Preprocessing <a name="loading"></a>

### Image Loading
The `loadImages` function handled:
- Loading images from the specified directory
- Converting from BGR to RGB color space
- Normalizing pixel values to [0, 1] range
- Creating grayscale versions for feature detection

In [ ]:
def loadImages(folder_path):
    """
    Load all images from a folder and convert them to float and grayscale.

    This function:
    - Loads up to max_images (from CONFIG) from the specified folder
    - Converts them to RGB float format (0-1 range)
    - Creates grayscale versions for feature detection

    Args:
        folder_path (str): Path to the folder containing images

    Returns:
        tuple: (list of RGB images, list of grayscale images)
    """
    # Get sorted list of image paths (limited by CONFIG)
    image_paths = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path)
                          if f.lower().endswith(('.jpg', '.jpeg', '.png'))])[:CONFIG['max_images']]

    images = []
    grays = []

    for i, path in enumerate(image_paths):
        img = cv2.imread(path)
        if img is None:
            continue

        # Convert BGR to RGB and normalize to [0, 1] range
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
        # Convert to grayscale for feature detection
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0

        images.append(img_rgb)
        grays.append(gray)

    print(f"Loaded {len(images)} images from: {folder_path}")
    return images, grays

### Exposure Compensation
Global exposure adjustment ensures consistent brightness across all images, which is crucial for seamless blending:
- Calculates mean brightness for each image
- Adjusts to match median brightness across all images
- Clips gain to prevent extreme adjustments (±30% maximum)

In [ ]:
def compensateExposure(images):
    """
    Perform global exposure adjustment across all images.

    This function:
    - Calculates the mean brightness of each image
    - Adjusts each image to match the median brightness
    - Clips the gain to prevent extreme adjustments

    Args:
        images (list): List of input images

    Returns:
        list: Exposure-adjusted images
    """
    # Calculate mean brightness for each image
    mean_brightness = [np.mean(img) for img in images]
    # Target brightness is the median across all images
    target_brightness = np.median(mean_brightness)

    adjusted_images = []
    for i, img in enumerate(images):
        # Calculate gain to reach target brightness
        gain = target_brightness / mean_brightness[i]
        # Limit gain to prevent extreme adjustments (30% max change)
        gain = np.clip(gain, 0.7, 1.3)
        adjusted_img = img * gain
        # Ensure pixel values stay in valid range [0, 1]
        adjusted_images.append(np.clip(adjusted_img, 0, 1))

    return adjusted_images

## 3. Feature Detection and Description <a name="features"></a>

### Multi-Scale Harris Corner Detection
Our enhanced feature detection implemented:
- **Multiple scales** (1.0, 1.2, 1.5) for better feature coverage
- **Bilateral filtering** for noise reduction while preserving edges
- **Grid-based selection** for uniform spatial distribution
- **Focused detection** reducing weight on bottom 35% (often less relevant areas)

### Harris Corner Response
The mathematical foundation:R = $det$(M) - $k·trace$($M$)²
Where:
- M is the structure tensor
- k = 0.04 (empirically determined constant)
- det(M) = λ₁λ₂ (product of eigenvalues)
- trace(M) = λ₁ + λ₂ (sum of eigenvalues)

In [ ]:
def getFeaturePoints(image, plot=True):
    """
    Enhanced multi-scale feature detection using Harris corner detector.

    Key innovations:
    - Multiple scales for better feature coverage
    - Bilateral filtering for noise reduction
    - Grid-based feature selection for spatial distribution
    - Focused detection on structural regions (reduced weight on bottom 35%)

    Args:
        image (np.array): Input image (RGB or grayscale)
        plot (bool): Whether to visualize detected features

    Returns:
        list: List of (x, y) corner coordinates
    """
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        gray = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY).astype(np.float32) / 255.0
    else:
        gray = image.copy()

    # Multi-scale detection for better feature coverage
    scales = [1.0, 1.2, 1.5]
    all_corners = []

    for scale in scales:
        # Resize image based on scale
        if scale != 1.0:
            scaled_gray = cv2.resize(gray, None, fx=1/scale, fy=1/scale)
        else:
            scaled_gray = gray

        # Apply bilateral filter to reduce noise while preserving edges
        filtered = cv2.bilateralFilter((scaled_gray * 255).astype(np.uint8), 9, 50, 50)
        scaled_gray = filtered.astype(np.float32) / 255.0

        # Compute image gradients using Sobel operator
        Ix = cv2.Sobel(scaled_gray, cv2.CV_32F, 1, 0, ksize=3)
        Iy = cv2.Sobel(scaled_gray, cv2.CV_32F, 0, 1, ksize=3)

        # Products of derivatives
        Ix2 = Ix * Ix
        Iy2 = Iy * Iy
        Ixy = Ix * Iy

        # Apply Gaussian window to compute weighted sums
        window_size = 5
        kernel = cv2.getGaussianKernel(window_size, 1.5)
        Sx2 = cv2.filter2D(Ix2, -1, kernel @ kernel.T)
        Sy2 = cv2.filter2D(Iy2, -1, kernel @ kernel.T)
        Sxy = cv2.filter2D(Ixy, -1, kernel @ kernel.T)

        # Harris corner response: R = det(M) - k*trace(M)^2
        k = 0.04  # Harris detector constant
        det = Sx2 * Sy2 - Sxy * Sxy
        trace = Sx2 + Sy2
        response = det - k * trace * trace

        # Reduce weight on bottom portion (often contains less relevant features)
        h, w = response.shape
        response[int(h*0.65):, :] *= 0.2

        # Threshold response to find corners
        threshold = 0.01 * response.max()
        corners = response > threshold

        # Non-maximum suppression to avoid clustered features
        local_maxima = maximum_filter(response, size=11)
        corners = corners & (response == local_maxima)

        # Extract corner coordinates and scale back to original size
        y_coords, x_coords = np.where(corners)
        for x, y in zip(x_coords, y_coords):
            all_corners.append((x * scale, y * scale, response[y, x]))

    # Sort corners by response strength
    all_corners.sort(key=lambda x: x[2], reverse=True)

    # Grid-based selection for better spatial distribution
    h, w = image.shape[:2]
    grid_size = CONFIG['feature_params']['grid_size']
    grid_corners = {}

    for x, y, resp in all_corners:
        grid_x = int(x / grid_size)
        grid_y = int(y / grid_size)
        key = (grid_x, grid_y)

        # Keep only the strongest corner in each grid cell
        if key not in grid_corners or resp > grid_corners[key][2]:
            grid_corners[key] = (x, y, resp)

    # Extract final corner points (limit to max_corners from CONFIG)
    corner_points = [(int(x), int(y)) for x, y, _ in grid_corners.values()][:CONFIG['feature_params']['max_corners']]

    if plot:
        plt.figure(figsize=(12, 8))
        plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
        if corner_points:
            points = np.array(corner_points)
            plt.scatter(points[:, 0], points[:, 1], c='red', s=15, marker='+')
        plt.title(f'Detected {len(corner_points)} corners')
        plt.axis('off')
        plt.show()

    return corner_points

### Feature Descriptors
Two descriptor types are supported:
1. **Gradient-based descriptors**
   - Creates 18-bin orientation histograms
   - Similar to SIFT descriptors
   - Weighted by gradient magnitude
2. **Patch-based descriptors**
   - Normalized intensity patches
   - Zero mean, unit variance normalization

In [ ]:
def getFeatureDescriptors(image, feature_points, mode='gradients', window_size=None):
    """
    Extract robust feature descriptors for matching.

    This function supports two modes:
    1. 'gradients': Creates orientation histograms (similar to SIFT)
    2. 'patches': Uses normalized intensity patches

    Args:
        image (np.array): Input image
        feature_points (list): List of (x, y) feature coordinates
        mode (str): Descriptor type ('gradients' or 'patches')
        window_size (int): Size of the window around each feature (uses CONFIG if None)

    Returns:
        tuple: (list of descriptors, list of valid feature points)
    """
    if window_size is None:
        window_size = CONFIG['feature_params']['window_size']

    descriptors = []
    valid_points = []
    half_window = window_size // 2
    h, w = image.shape[:2]

    if mode == 'gradients':
        # Compute image gradients
        dx = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=3)
        dy = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=3)
        magnitude = np.sqrt(dx**2 + dy**2)
        orientation = np.arctan2(dy, dx)

        for x, y in feature_points:
            # Check if window is within image bounds
            if (y >= half_window and y < h - half_window and
                x >= half_window and x < w - half_window):

                # Extract local windows
                mag_win = magnitude[y-half_window:y+half_window, x-half_window:x+half_window]
                ori_win = orientation[y-half_window:y+half_window, x-half_window:x+half_window]

                # Create orientation histogram (18 bins covering -π to π)
                hist = np.zeros(18)
                bins = np.linspace(-np.pi, np.pi, 19)

                # Accumulate weighted orientations
                for r in range(mag_win.shape[0]):
                    for c in range(mag_win.shape[1]):
                        bin_idx = np.digitize(ori_win[r, c], bins) - 1
                        if 0 <= bin_idx < 18:
                            # Weight by gradient magnitude
                            hist[bin_idx] += mag_win[r, c]

                # Normalize histogram
                if hist.sum() > 0:
                    hist = hist / hist.sum()

                descriptors.append(hist)
                valid_points.append((x, y))
    else:
        # Patch-based descriptors
        for x, y in feature_points:
            if (y >= half_window and y < h - half_window and
                x >= half_window and x < w - half_window):

                # Extract and flatten patch
                window = image[y-half_window:y+half_window, x-half_window:x+half_window]
                descriptor = window.flatten()

                # Normalize descriptor (zero mean, unit variance)
                descriptor = descriptor - descriptor.mean()
                if descriptor.std() > 0:
                    descriptor = descriptor / descriptor.std()

                descriptors.append(descriptor)
                valid_points.append((x, y))

    return descriptors, valid_points

##  4. Advanced Feature Matching <a name="matching"></a>

### Distance Metrics Implementation

#### 1) Chi-squared Distance
Particularly effective for comparing probability distributions:
$χ$² = 0.5 × $Σ$$(h₁ - h₂)²$ / $(h₁ + h₂))$
#### 2) Bhattacharyya Distance
Measures overlap between two distributions:
D = -$ln$$(BC)$ where $BC$ = $Σ$$√(h₁ × h₂)$
#### 3) Earth Mover's Distance (Wasserstein)
Computes minimum cost to transform one distribution into another:
- Robust to small shifts in distributions
- Considers the "effort" needed to match histograms

#### 4)Attention-based Matching
Inspired by transformer architectures:
1. Computes similarity matrix using dot products
2. Applies softmax with temperature control
3. Uses attention weights to find globally consistent matches
4. Temperature parameter controls matching selectivity

### 5) Ratio Test
Implements Lowe's ratio test for robust matching:
- Compares best match distance to second-best
- Rejects ambiguous matches
- Threshold typically set to 0.7-0.8

In [ ]:
def chi_squared_distance(hist1, hist2):
    """
    Chi-squared distance for histogram comparison.

    Chi-squared distance is particularly effective for comparing
    probability distributions like orientation histograms.

    Formula: χ² = 0.5 * Σ((h1 - h2)² / (h1 + h2))
    """
    return 0.5 * np.sum((hist1 - hist2)**2 / (hist1 + hist2 + 1e-10))

def bhattacharyya_distance(hist1, hist2):
    """
    Bhattacharyya distance for histogram similarity.

    This metric measures the similarity between two probability distributions.
    It's related to the Bhattacharyya coefficient.

    Formula: D = -ln(BC) where BC = Σ√(h1 * h2)
    """
    bc = np.sum(np.sqrt(hist1 * hist2))
    return -np.log(bc + 1e-10)

def earth_movers_distance(hist1, hist2):
    """
    Earth Mover's Distance (Wasserstein distance).

    EMD computes the minimum cost of transforming one distribution
    into another, making it robust to small shifts.
    """
    return wasserstein_distance(range(len(hist1)), range(len(hist2)), hist1, hist2)

def attention_based_matching(desc1_all, desc2_all, threshold=0.3, temperature=0.1):
    """
    Attention-based matching with global context.

    This method uses attention mechanisms similar to transformers
    to find correspondences by considering global feature relationships.

    Args:
        desc1_all (np.array): Descriptors from image 1
        desc2_all (np.array): Descriptors from image 2
        threshold (float): Minimum similarity threshold
        temperature (float): Softmax temperature (lower = sharper attention)

    Returns:
        list: Matched feature pairs [(idx1, idx2), ...]
    """
    # Compute similarity matrix (dot product for cosine similarity)
    similarity_matrix = desc1_all @ desc2_all.T

    # Apply softmax with temperature to get attention weights
    attention_weights = np.exp(similarity_matrix / temperature)
    attention_weights /= attention_weights.sum(axis=1, keepdims=True)

    # Find best matches using attention-weighted similarities
    matches = []
    used_j = set()

    for i in range(len(desc1_all)):
        # Weight similarities by attention
        weighted_similarities = attention_weights[i] * similarity_matrix[i]
        # Sort by weighted similarity
        sorted_indices = np.argsort(weighted_similarities)[::-1]

        # Find best unused match
        for j in sorted_indices:
            if j not in used_j and weighted_similarities[j] > threshold:
                matches.append((i, j))
                used_j.add(j)
                break

    return matches

Main Matching Function


In [ ]:
def match2Images(points1, descriptors1, points2, descriptors2,
                 threshold=0.7, distance_type='chi_squared', plot=True):
    """
    Enhanced matching with multiple distance metrics.

    This function supports various distance metrics for finding
    correspondences between feature descriptors from two images.

    Supported metrics:
    - 'euclidean': Standard L2 distance
    - 'correlation': Correlation coefficient
    - 'chi_squared': Chi-squared distance for histograms
    - 'bhattacharyya': Bhattacharyya distance
    - 'earth_movers': Earth Mover's Distance
    - 'attention': Attention-based matching

    Args:
        points1, points2: Feature point coordinates
        descriptors1, descriptors2: Feature descriptors
        threshold: Distance threshold for matches
        distance_type: Type of distance metric to use
        plot: Whether to visualize matches

    Returns:
        list: Matched pairs [(idx1, idx2), ...]
    """
    if not descriptors1 or not descriptors2:
        return []

    desc1 = np.array(descriptors1)
    desc2 = np.array(descriptors2)

    # Special handling for attention-based matching
    if distance_type == 'attention':
        matches = attention_based_matching(desc1, desc2, threshold)
        print(f"Found {len(matches)} matches using attention")
        return matches

    # Compute distance matrix for other metrics
    distances = np.zeros((len(desc1), len(desc2)))

    if distance_type == 'euclidean':
        distances = cdist(desc1, desc2, 'euclidean')
    elif distance_type == 'correlation':
        for i in range(len(desc1)):
            for j in range(len(desc2)):
                corr = np.corrcoef(desc1[i], desc2[j])[0, 1]
                distances[i, j] = 1 - abs(corr)
    elif distance_type == 'chi_squared':
        for i in range(len(desc1)):
            for j in range(len(desc2)):
                distances[i, j] = chi_squared_distance(desc1[i], desc2[j])
    elif distance_type == 'bhattacharyya':
        for i in range(len(desc1)):
            for j in range(len(desc2)):
                distances[i, j] = bhattacharyya_distance(desc1[i], desc2[j])
    elif distance_type == 'earth_movers':
        for i in range(len(desc1)):
            for j in range(len(desc2)):
                distances[i, j] = earth_movers_distance(desc1[i], desc2[j])

    # Apply ratio test (Lowe's ratio test)
    matches = []
    for i in range(len(desc1)):
        sorted_idx = np.argsort(distances[i])
        best = sorted_idx[0]
        second_best = sorted_idx[1]

        # Check if best match is significantly better than second best
        if distances[i, best] < threshold * distances[i, second_best]:
            matches.append((i, best))

    if plot and matches:
        plt.figure(figsize=(12, 5))
        plt.subplot(121)
        pts1 = np.array([points1[m[0]] for m in matches[:50]])
        plt.scatter(pts1[:, 0], pts1[:, 1], c='red', s=10)
        plt.title(f'Image 1 - {len(matches)} matches ({distance_type})')

        plt.subplot(122)
        pts2 = np.array([points2[m[1]] for m in matches[:50]])
        plt.scatter(pts2[:, 0], pts2[:, 1], c='blue', s=10)
        plt.title('Image 2')
        plt.show()

    print(f"Found {len(matches)} matches using {distance_type}")
    return matches

## 5. Homography Estimation and RANSAC <a name="ransac"></a>

### RANSAC Algorithm
Random Sample Consensus for robust homography estimation:

1. **Random Sampling**: Select 4 point correspondences
2. **Homography Calculation**: Compute H using Direct Linear Transform (DLT)
3. **Inlier Detection**: Count points with reprojection error < threshold
4. **Iteration**: Repeat and keep best homography

### Adaptive Threshold
Our implementation features adaptive inlier threshold:
- Analyzes motion magnitude statistics
- Sets threshold to 75th percentile of motion magnitudes
- Prevents overly strict or loose criteria

### Direct Linear Transform (DLT)
For each correspondence (x,y) → (u,v):

In [ ]:
def refineMatches(points1, points2, matches, inlier_threshold=4.0, iterations=2000, plot=True):
    """
    RANSAC-based homography estimation with adaptive threshold.

    RANSAC process:
    1. Randomly select 4 matches (minimum for homography)
    2. Compute homography from these 4 points
    3. Count inliers based on reprojection error
    4. Repeat and keep best homography

    Args:
        points1, points2: Feature coordinates in both images
        matches: Initial matches [(idx1, idx2), ...]
        inlier_threshold: Maximum error for inliers (pixels)
        iterations: Number of RANSAC iterations
        plot: Whether to visualize inlier distribution

    Returns:
        tuple: (best_homography, inlier_indices)
    """
    if len(matches) < 4:
        return None, []

    # Convert matches to point correspondences
    pts1 = np.float32([points1[m[0]] for m in matches])
    pts2 = np.float32([points2[m[1]] for m in matches])

    # Adaptive threshold based on motion statistics
    motion_magnitudes = np.linalg.norm(pts2 - pts1, axis=1)
    adaptive_threshold = min(inlier_threshold, np.percentile(motion_magnitudes, 75))

    best_H = None
    best_inliers = []

    for _ in range(iterations):
        # Randomly select 4 matches
        idx = np.random.choice(len(matches), 4, replace=False)
        src_pts = pts1[idx]
        dst_pts = pts2[idx]

        # Build linear system for homography estimation
        # Using Direct Linear Transform (DLT)
        A = []
        for i in range(4):
            x, y = src_pts[i]
            u, v = dst_pts[i]
            # Each correspondence gives 2 equations
            A.append([-x, -y, -1, 0, 0, 0, u*x, u*y, u])
            A.append([0, 0, 0, -x, -y, -1, v*x, v*y, v])

        A = np.array(A)
        try:
            # Solve using SVD
            _, _, V = np.linalg.svd(A)
            H = V[-1].reshape(3, 3)
            H = H / H[2, 2]  # Normalize so H[2,2] = 1

            # Count inliers
            inliers = []
            for i in range(len(pts1)):
                # Apply homography
                pt = np.append(pts1[i], 1)
                transformed = H @ pt
                if transformed[2] != 0:
                    transformed = transformed[:2] / transformed[2]
                    # Calculate reprojection error
                    error = np.linalg.norm(transformed - pts2[i])
                    if error < adaptive_threshold:
                        inliers.append(i)

            # Update best if this has more inliers
            if len(inliers) > len(best_inliers):
                best_H = H
                best_inliers = inliers
        except:
            continue

    # Print statistics
    if best_inliers:
        residuals = []
        for i in best_inliers:
            pt = np.append(pts1[i], 1)
            transformed = best_H @ pt
            transformed = transformed[:2] / transformed[2]
            residuals.append(np.linalg.norm(transformed - pts2[i]))
        avg_residual = np.mean(residuals)
        print(f"RANSAC: {len(best_inliers)} inliers, avg residual: {avg_residual:.2f}")

    if plot and best_inliers:
        plt.figure(figsize=(10, 5))
        inlier_mask = np.zeros(len(matches), dtype=bool)
        for i in best_inliers:
            inlier_mask[i] = True
        plt.bar(range(len(matches)), inlier_mask.astype(int))
        plt.title(f'{len(best_inliers)} inliers after RANSAC')
        plt.ylabel('Inlier')
        plt.xlabel('Match index')
        plt.show()

    return best_H, best_inliers

## 6. Image Warping and Blending <a name="warping"></a>

### Warping Pipeline
1. **Exposure compensation** for consistent brightness
2. **Cumulative transformation** calculation from center image
3. **Canvas size** determination from transformed corners
4. **Progressive warping** starting from center
5. **Smooth blending** of overlapping regions

### Advanced Blending Technique
Our blending method uses distance transforms:

1. **Distance Maps**: Calculate distance from mask edges
2. **Weight Calculation**: Proportional to distance from edge
3. **Overlap Handling**: Smooth transitions in overlap regions
4. **Gaussian Smoothing**: Final smoothing of blend weights
5. **Per-channel Blending**: Separate blending for R, G, B channels

### Mathematical Formulation
For overlapping regions:

In [ ]:
def smoothBlend(img1, img2, mask1, mask2):
    """
    Smooth blending with feathering for seamless transitions.

    This function uses distance transforms to create smooth
    blend weights in overlapping regions.

    Args:
        img1, img2: Images to blend
        mask1, mask2: Binary masks indicating valid pixels

    Returns:
        np.array: Blended image
    """
    # Create distance maps from mask edges
    dist1 = cv2.distanceTransform((mask1 > 0).astype(np.uint8), cv2.DIST_L2, 5)
    dist2 = cv2.distanceTransform((mask2 > 0).astype(np.uint8), cv2.DIST_L2, 5)

    # Normalize distances
    dist1 = dist1 / (dist1.max() + 1e-5)
    dist2 = dist2 / (dist2.max() + 1e-5)

    # Create blend weights
    overlap = mask1 * mask2
    blend_weight = np.zeros_like(mask1)

    # Smooth transition in overlap regions
    overlap_indices = np.where(overlap > 0)
    if len(overlap_indices[0]) > 0:
        weights1 = dist1[overlap_indices]
        weights2 = dist2[overlap_indices]
        # Weight proportional to distance from edge
        blend_weight[overlap_indices] = weights1 / (weights1 + weights2 + 1e-6)

    # Non-overlap regions get full weight
    blend_weight[mask1 > mask2] = 1
    blend_weight[mask2 > mask1] = 0

    # Apply Gaussian blur for smoother transitions
    blend_weight = cv2.GaussianBlur(blend_weight, (31, 31), 10)

    # Blend images channel by channel
    result = np.zeros_like(img1)
    for c in range(3):
        result[:, :, c] = img1[:, :, c] * blend_weight + img2[:, :, c] * (1 - blend_weight)

    return result

Complete Warping Pipeline

In [ ]:
def warpImages(images, homographies, plot=True):
    """
    Warp all images to create the final panorama.

    This function:
    1. Applies exposure compensation
    2. Computes cumulative transformations from center
    3. Calculates output canvas size
    4. Warps and blends images progressively

    Args:
        images: List of input images
        homographies: List of pairwise homographies
        plot: Whether to display the result

    Returns:
        np.array: Final panorama
    """
    n = len(images)

    print(f"Creating panorama with all {n} images...")

    # Pre-process images for consistent exposure
    adjusted_images = compensateExposure(images)

    # Calculate cumulative transformations from center
    center_idx = n // 2
    H_cumulative = [None] * n
    H_cumulative[center_idx] = np.eye(3)

    # Build transformations going left from center
    current_H = np.eye(3)
    for i in range(center_idx - 1, -1, -1):
        if i < len(homographies) and homographies[i] is not None:
            # Chain transformations
            current_H = current_H @ np.linalg.inv(homographies[i])
            H_cumulative[i] = current_H
        else:
            H_cumulative[i] = None

    # Build transformations going right from center
    current_H = np.eye(3)
    for i in range(center_idx, n - 1):
        if i < len(homographies) and homographies[i] is not None:
            # Chain transformations
            current_H = current_H @ homographies[i]
            H_cumulative[i + 1] = current_H
        else:
            H_cumulative[i + 1] = None

    # Find valid indices (images with valid homographies)
    valid_indices = [i for i in range(n) if H_cumulative[i] is not None]

    if len(valid_indices) < 2:
        return images[center_idx]

    # Calculate output canvas size
    all_corners = []
    for idx in valid_indices:
        h, w = adjusted_images[idx].shape[:2]
        # Define corner points
        corners = np.array([[0, 0, 1], [w, 0, 1], [w, h, 1], [0, h, 1]]).T

        # Transform corners
        warped_corners = H_cumulative[idx] @ corners
        warped_corners = warped_corners[:2] / warped_corners[2]
        all_corners.append(warped_corners.T)

    # Find bounding box
    all_corners = np.vstack(all_corners)
    min_xy = np.floor(all_corners.min(axis=0)).astype(int)
    max_xy = np.ceil(all_corners.max(axis=0)).astype(int)

    out_width = max_xy[0] - min_xy[0]
    out_height = max_xy[1] - min_xy[1]

    # Limit output size to prevent memory issues
    max_width = 10000
    max_height = 5000
    if out_width > max_width or out_height > max_height:
        scale = min(max_width/out_width, max_height/out_height)
        out_width = int(out_width * scale)
        out_height = int(out_height * scale)

    # Translation to ensure all pixels are positive
    translation = np.array([[1, 0, -min_xy[0]], [0, 1, -min_xy[1]], [0, 0, 1]])

    # Start with center image
    center_H = translation @ H_cumulative[center_idx]
    result = cv2.warpPerspective(
        (adjusted_images[center_idx] * 255).astype(np.uint8),
        center_H.astype(np.float32),
        (out_width, out_height)
    )
    result = result.astype(np.float32) / 255.0

    # Add remaining images, starting from closest to center
    indices_by_distance = sorted(valid_indices, key=lambda x: abs(x - center_idx))

    for idx in indices_by_distance:
        if idx == center_idx:
            continue

        # Warp current image
        H_total = translation @ H_cumulative[idx]

        warped = cv2.warpPerspective(
            (adjusted_images[idx] * 255).astype(np.uint8),
            H_total.astype(np.float32),
            (out_width, out_height)
        )
        warped = warped.astype(np.float32) / 255.0

        # Create masks for blending
        mask_result = (result.sum(axis=2) > 0).astype(float)
        mask_warped = (warped.sum(axis=2) > 0).astype(float)

        # Smooth blending
        result = smoothBlend(result, warped, mask_result, mask_warped)

        print(f"Added image {idx + 1}")

    if plot:
        plt.figure(figsize=(20, 10))
        plt.imshow(result)
        plt.title(f'Final Panorama (All {len(valid_indices)} images)')
        plt.axis('off')
        plt.show()

    return result

7. Main Execution and Results


In [ ]:
def main():
    """
    Main execution pipeline for panorama creation.

    Steps:
    1. Load all images
    2. Detect features in all images
    3. Match features using multiple metrics
    4. Estimate homographies with RANSAC
    5. Create final panorama
    """
    folder = CONFIG['folder_path']

    # Step 1: Load all images
    print("Step 1: Loading all images...")
    images, grays = loadImages(folder)

    # Step 2: Feature detection
    print("\nStep 2: Detecting features in all images...")
    all_points = []
    all_descriptors = []

    for i, gray in enumerate(grays):
        print(f"Processing image {i+1}/{len(grays)}")
        points = getFeaturePoints(gray, plot=(i==0))
        descriptors, valid_points = getFeatureDescriptors(gray, points, mode='gradients')
        all_points.append(valid_points)
        all_descriptors.append(descriptors)
        print(f"Image {i+1}: {len(valid_points)} features")

    # Step 3: Feature matching with different metrics
    print("\nStep 3: Matching features with advanced metrics...")
    homographies = []

    # Distance metrics in order of preference
    distance_metrics = ['chi_squared', 'bhattacharyya', 'attention', 'euclidean']

    for i in range(len(images)-1):
        print(f"\nMatching images {i+1} and {i+2}:")

        best_matches = []
        best_metric = None

        # Try different distance metrics
        for metric in distance_metrics:
            try:
                threshold = CONFIG['matching_params']['attention_threshold'] if metric == 'attention' else CONFIG['matching_params']['threshold']
                matches = match2Images(
                    all_points[i], all_descriptors[i],
                    all_points[i+1], all_descriptors[i+1],
                    threshold=threshold,
                    distance_type=metric,
                    plot=False
                )

                if len(matches) > len(best_matches):
                    best_matches = matches
                    best_metric = metric

            except Exception as e:
                print(f"Failed with {metric}: {e}")
                continue

        print(f"Best metric: {best_metric} with {len(best_matches)} matches")

        # RANSAC refinement
        if len(best_matches) >= 10:
            H, inliers = refineMatches(
                all_points[i], all_points[i+1], best_matches,
                inlier_threshold=CONFIG['ransac_params']['inlier_threshold'],
                iterations=CONFIG['ransac_params']['iterations'],
                plot=(i==0)
            )
            if H is not None and len(inliers) >= 20:
                homographies.append(H)
                print(f"Homography estimated with {len(inliers)} inliers")
            else:
                homographies.append(None)
                print("Failed to estimate reliable homography")
        else:
            homographies.append(None)
            print("Too few matches")

    # Step 4: Create panorama
    print("\nStep 4: Creating panorama...")
    panorama = warpImages(images, homographies)

    # Save result
    output_path = os.path.join(folder, CONFIG['output_filename'])
    cv2.imwrite(output_path, (panorama * 255).astype(np.uint8)[:, :, ::-1])
    print(f"\nPanorama saved to: {output_path}")

    return panorama

# Run the main function
if __name__ == "__main__":
    panorama = main()

## 🎉 Conclusion

This panorama creation pipeline demonstrates the effectiveness of using multiple distance metrics for feature matching. The combination of:
- Advanced feature detection
- Multiple matching strategies
- Robust homography estimation
- Sophisticated blending techniques

produces high-quality panoramas even with challenging image sets. The modular design allows easy experimentation with different components and parameters.